# Smart case study

## import libraly

In [1]:
import gdown
import pandas as pd

## Dowload dataset from google disk

In [2]:
# URL to the file on Google Drive
url_rents = 'https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x'
output_rents = '../data/snp_dld_2024_rents.csv'
url_trans = 'https://drive.google.com/uc?id=1liNykIOnfR5KRR4MXJISCZCKJYFnXQC1'
output_trans = '../data/snp_dld_2024_transactions.csv'
# Download the file
gdown.download(url_rents, output_rents, quiet=False)
gdown.download(url_trans, output_trans, quiet=False)

# Load data into DataFrame
snp_dld_2024_rents = pd.read_csv(output_rents)
snp_dld_2024_transactions = pd.read_csv(output_trans)
print(snp_dld_2024_rents.shape)
print(snp_dld_2024_transactions.shape)

Downloading...
From (original): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x
From (redirected): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x&confirm=t&uuid=b4d2d34c-0881-4ea1-a7a6-4411e8f65004
To: /workspaces/SB-Case-Study-Materials/data/snp_dld_2024_rents.csv
100%|██████████| 278M/278M [00:05<00:00, 50.9MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1liNykIOnfR5KRR4MXJISCZCKJYFnXQC1
To: /workspaces/SB-Case-Study-Materials/data/snp_dld_2024_transactions.csv
100%|██████████| 70.1M/70.1M [00:01<00:00, 59.2MB/s]
/tmp/ipykernel_77543/1945715564.py:11: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  snp_dld_2024_rents = pd.read_csv(output_rents)


(681447, 43)
(162806, 46)


## Preprocess

In [3]:
rentals = pd.read_csv('../data/snp_dld_2024_rents.csv')

/tmp/ipykernel_77543/3829352147.py:1: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  rentals = pd.read_csv('../data/snp_dld_2024_rents.csv')


In [4]:
rentals[['registration_date', 'contract_start_date', 'contract_end_date', 'req_from', 'req_to']] = rentals[['registration_date', 'contract_start_date', 'contract_end_date', 'req_from', 'req_to']].apply(pd.to_datetime)

In [5]:
rentals['is_freehold'] = rentals['is_freehold'].replace({'t': True, 'f': False})
rentals['delta_time_registration'] = (rentals['registration_date'] - rentals['contract_start_date']).dt.days
rentals['delta_time_reg_from'] = (rentals['registration_date'] - rentals['req_from']).dt.days

rentals['time_reg'] = (rentals['req_to'] - rentals['req_from']).dt.days
rentals['time_contract'] = (rentals['contract_end_date'] - rentals['contract_start_date']).dt.days

rentals.sort_values('version_number', ascending=False, inplace=True)
rentals = rentals.drop_duplicates(subset=['ejari_contract_number', 'registration_date', 'contract_start_date', 'contract_end_date', 'contract_amount', 'property_type_en'], keep='last')
rentals.query('contract_amount > 0', inplace=True)


/tmp/ipykernel_77543/424522771.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rentals['is_freehold'] = rentals['is_freehold'].replace({'t': True, 'f': False})


In [6]:
rentals.drop(['is_freehold_text', 'land_property_id', 'property_id', 'property_type_ar', 'property_subtype_ar', 
              'property_usage_ar', 'property_usage_id', 'project_name_ar', 'area_ar', 'nearest_landmark_ar', 
              'nearest_metro_ar', 'nearest_mall_ar', 'master_project_ar', 'ejari_property_type_id', 
              'ejari_property_sub_type_id', 'meta_ts', 'area_id', 'version_number',
              'contract_end_date', 'contract_start_date', 'registration_date', 'req_from', 'req_to'
              ], axis=1, inplace=True)


In [7]:
rentals

,ejari_contract_number,version_text,contract_amount,annual_amount,is_freehold,property_size_sqm,parcel_id,property_type_en,property_subtype_en,property_usage_en,...,area_en,nearest_landmark_en,nearest_metro_en,nearest_mall_en,master_project_en,entry_id,delta_time_registration,delta_time_reg_from,time_reg,time_contract
585389,1469528750,Renewed,38520.0,68561.91,False,165.00,1340948.0,Unit,Flat,Residential,...,Al Mamzer,Dubai International Airport,STADIUM Metro Station,City Centre Mirdif,NaN,94374,25,3,29,205
426295,1469528750,Renewed,11220.0,68423.61,False,165.00,1340948.0,Unit,Flat,Residential,...,Al Mamzer,Dubai International Airport,STADIUM Metro Station,City Centre Mirdif,NaN,94372,26,5,30,59
586703,73768715,Renewed,23621.0,94484.00,False,130.11,3360234.0,Unit,Flat,Residential,...,Trade Center Second,Burj Khalifa,Buj Khalifa Dubai Mall Metro Station,Dubai Mall,NaN,94374,11,25,29,91
425814,73768715,Renewed,23364.0,93456.00,False,130.11,3360234.0,Unit,Flat,Residential,...,Trade Center Second,Burj Khalifa,Buj Khalifa Dubai Mall Metro Station,Dubai Mall,NaN,94372,16,0,30,90
96278,47959553,Renewed,65000.0,65000.00,False,107.86,3360136.0,Unit,Flat,Residential,...,Trade Center Second,Burj Khalifa,Financial Centre,Dubai Mall,NaN,94368,33,2,30,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361435,120240501002395,New,21000.0,21000.00,False,20.25,1230579.0,Unit,Office,Commercial,...,Al Muteena,Dubai International Airport,Salah Al Din Metro Station,Dubai Mall,NaN,94372,0,0,30,364
362333,120240501002396,New,43000.0,43000.00,False,35.00,3140132.0,Unit,Flat,Residential,...,Um Hurair First,Dubai International Airport,Burjuman Metro Station,Dubai Mall,NaN,94372,0,0,30,364
362390,120240501002403,New,22000.0,22000.00,False,22.00,1170113.0,Unit,Office,Commercial,...,Al Mararr,Dubai International Airport,Baniyas Square Metro Station,Dubai Mall,NaN,94372,0,0,30,364
362906,120240501002404,New,115000.0,115000.00,True,100.40,3260888.0,Unit,Flat,Residential,...,Al Jadaf,Dubai International Airport,Al Jadaf Metro Station,Dubai Mall,NaN,94372,0,0,30,364
